# Exploring Information Theory in Experimental Design

Welcome to this tutorial on the basics of information theory and its practical application in optimizing experimental design! In this session, we'll delve into the fundamental concepts of information content, entropy, conditional entropy, and mutual information, all within the context of a synthetic working memory experiment.

#### Why should you even bother about Information Theory when designing an experiment?

Imagine you're designing an experiment to understand how human working memory performs under conditions like the amount of information to remember. How do you choose the amount participants should remember? How do you quantify the information gained from each trial about this cognitive system under study? This is where information theory steps in.

Information theory provides a powerful framework for measuring and quantifying uncertainty, information content, and the relationships between variables. In this tutorial, we'll guide you through the basics and show you how to apply these concepts in experimental design.

#### Optimizing the experimental design for a working memory experiment

Our context for learning will be a synthetic working memory experiment. 

In this experiment, participants (experimental units) are asked to remember a sequence of the length *L* which contains random numbers. 

We, as the researchers, have control about the sequence length *L*. This is the factor in our experiment.

The participants response will be categorized either being *Correct* (1) or *False* (0).

#### Tutorial Overview

In this tutorial you will learn to:
- Set up a synthetic working memory experiment (remember what you learned in the tutorial about synthetic experiments)
- Compute the information content and the entropy of the responses
- Choose new factor levels based on our knowledge of the entropy
- Compute the mutual information between our model parameters, the factors and the given responses

## Importing libraries

In [1]:
import os, sys

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# Set the path to the project folder
target_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust path as needed
if target_folder not in sys.path:
    sys.path.append(target_folder)

## Setup of the synthetic experiment

In the first section, we are going to set up the synthetic experiment by defining the experimental unit.

You can think of each unit being one participant.

Remember what you learned in the tutorial about synthetic experimentation where you implemented everything by yourselves.

But instead of implementing everything on our own, we just take the already implemented experimental_unit class, a ground truth and a noise method. You can find these elements in the `synthetic.py` file.

Import the synthetic file with all components.

In [2]:
from resources.synthetic import experimental_unit, sigmoid_ground_truth, noise

The ground truth method synthetic.sigmoid_ground_truth will give you a curve of sigmoidal shape as you know it already from the presented working memory experiment from the lecture.

In this case the ground truth does not give directly the response because it just tells you the probability for remembering the sequence correctly for a sequence of sequence length *L* under a certain parameter *p*.

We have to wrap it first into a method that transforms these probabilities into a binary response (remember: 0 = false; 1 = correct).

In [3]:
# make a wrapper for the ground truth method
# this is needed to transform the probability given by the ground truth into a binary response (0: False or 1: Correct)

def ground_truth_wrapper(conditions, parameters, *args):
    # give a random response based on the probability given by the ground truth
    # get a random variable between 0 and 
    
    # make sure that the inputs of the wrapper are same ones as for the ground truth
    probability = sigmoid_ground_truth(conditions, parameters)
    # define a random variable which we will check against the probability given by the ground truth to decide if we return 1 or 0
    random_variable = np.random.uniform(0, 1)
    # check if the random variable is smaller than the probability given by the ground truth -> return 1; else return 0
    # example: if the probability given by the ground truth is 0.7, we return 1 in 70% of the cases and 0 in 30% of the cases
    return (random_variable < probability).astype(int)

Let's test the wrapper and see what it actually does!

First, we are going to define the maximum sequence length for our experiment.

Then we set up one participant by simply defining a parameter and check the probabilities of answering correctly - in the real world case we can't observe that probability directly.

In [4]:
maximum_sequence_length = 20

In [ ]:
sequence_lengths = np.arange(1, maximum_sequence_length)  # sequence lengths for which we want to know the probability of correct answer
parameters = (7, -1)  # participant parameter

# collect the probabilities for each sequence length
probs = np.zeros(len(sequence_lengths))
for i, l in enumerate(sequence_lengths):
    probs[i] = sigmoid_ground_truth(l, parameters)

# plot the ground truth probability curve    
plt.plot(sequence_lengths, probs, '-o')
plt.ylabel('Probability of correct answer')
plt.xlabel('Sequence length')
plt.title('Probability of this participant answering correctly for a sequence of length L')
plt.show()

How does your graph look like? Does it resemble the sigmoidal shape with a 50% probability of answering correctly around the sequence length 7?

If not, you may want to adjust your model parameters.

Now we are going to take this ground truth and wrap it into the previously defined method that actually returns a binary response based on the response probability.

In [ ]:
l = 6 # sequence length
for i in range(10):
    binary_response = ground_truth_wrapper(l, parameters)
    print(f'The given response is: {binary_response};\tThe actual probability of correct answer is: {np.round(sigmoid_ground_truth(l, parameters), 2)}')

According to probability theory, our wrapper should now approximate the correct probabilities for $n_\text{repitions}\rightarrow\inf$.

Let's check how our knowledge of the model reflects real circumstances based on the amount of repetitions per condition (sequence length).

First, we are going to collect the responses of the defined participant for each sequence length $n_\text{repitions}$ times.

We are going to do $n_\text{experiments}$ independent repetitions of the experiment to show how the experiment-to-experiment variance and therefore our certainty/confidence about the resulting model changes with the amount of repitions 

In [ ]:
n_repetitions = 1  # number of repetitions for each sequence length

n_experiments = 100  # number of repetitions for the entire experiment to visualize difference in obtained results

# do not change the code from this point
np.random.seed(42) 
binary_response = np.zeros((n_experiments, len(sequence_lengths), n_repetitions))
found_probs = np.zeros((n_experiments, len(sequence_lengths)))

# collect the responses from the wrapper method
# we are going to repeat this 10 times for each sequence length
for e in range(n_experiments):
    for i, l in enumerate(sequence_lengths):
        for r in range(n_repetitions):
            binary_response[e, i, r] = ground_truth_wrapper(l, parameters)

found_probs = np.sum(binary_response, axis=-1)/n_repetitions

# plot the ground truth probability curve 
for e in range(n_experiments):
    plt.plot(sequence_lengths, found_probs[e], '--', alpha=0.3, color='tab:orange')
plt.plot(sequence_lengths, probs, '-o', color='tab:blue')
plt.ylabel('Probability of correct answer')
plt.xlabel('Sequence length')
plt.title('Probability of this participant answering correctly for a sequence of length L')
plt.show()

How does the result look like? Change the number of repetitions to see how the found probability approximates the actual one.

In the next step we will make a bunch of experimental units with the implemented wrapper function and the set of parameters defined in the beginning.

Unlike the last tutorial for synthetic experimentation, we won't need any noise this time because our wrapper gives already a randomized response by generating a random number which is simply checked against the given probabilities.

### Collecting the data

In this section we are going to set up our data collection and generate the data.

In [8]:
# define the dataset parameters

# number of experimental units
n_units = 20

# sequence lengths which we are going to use
sequence_lengths = np.arange(1, 100).reshape(-1, 1)

# number of repetitions per condition
n_repetitions = 1

In [ ]:
# make a list of parameters which are passed to the experimental_unit class
parameter_0 = np.random.normal(7, 0.5, (n_units, 1))
parameter_1 = np.random.normal(-1, 0.25, (n_units, 1))
parameters = np.concatenate((parameter_0, parameter_1), axis=1)

# defining an experimental unit with the wrapper method
experimental_units = []
for p in parameters:
    experimental_units.append(
        experimental_unit(
            problem_solver=ground_truth_wrapper, 
            noise=noise, 
            parameters=p, 
            noise_level=0
            )
        )

print(f'Number of experimental units in the dataset: {len(experimental_units)}')
test_sequence_length = 7
if experimental_units[0].step(test_sequence_length)[0] == 1:
    print(f'Given a sequence length of {test_sequence_length}, one of the experimental units repeated the sequence successfully')
else:
    print(f'Given a sequence length of {test_sequence_length}, one of the experimental units failed to repeat the sequence')

Does your setup look good?

If that's the case, great! Then we can collect the data.


The dataset is a 4D tensor with the following dimensions:
1. number of experimental units
2. number of sequence lengths
3. number of repetitions
4. features -> unit_id, sequence_length, observation

You can use the `generate_dataset` method, which you should already know from the previous tutorials.

In [10]:
from resources.synthetic import generate_dataset

dataset, dataset_flat = generate_dataset(experimental_units, sequence_lengths, n_repetitions, True)

conditions = dataset[:, :, :, 1]
observations = dataset[:, :, :, -1]

Let's check how our dataset looks like

In [ ]:
# compute the mean of the observations to get the probability for answering correctly for each sequence_length and for each participant
probability = np.mean(observations, axis=-1, keepdims=True)

# plot the ground truth probability curve 
for i, unit in enumerate(probability):
    plt.plot(conditions[i, :, 0], unit, '-', alpha=0.1, color='tab:orange')
# plt.plot(sequence_lengths, probs, '-o', color='tab:blue')
plt.ylabel('Probability of correct answer')
plt.xlabel('Sequence length')
plt.title('Probability of each participant answering correctly\nfor each sequence length\n(averaged across repitions)')
plt.show()

print(f'The shape of the dataset is: {dataset.shape}')
print(dataset)

Does it look good? 

Then you have successfully finished the experimental setup and we can move on to the application of the information theoretic principles you learned about in the lecture.

## Applied information theory for experimental design

In this section you will apply and compute the information theoretic concepts you learned about in the lecture.

This includes information content, entropy, conditional entropy and mutual information.

You will learn about:
- the information content of obtained samples
- the overall entropy of the whole dataset
- the conditional entropy i.e., the remaining entropy of the responses when the condition is given
- the mutual information between the respones, the sequence lengths and a model which we are going to set up

### Information content

In this section we are going to compute the information content of a few samples first in general and then based on the sequence length.

To do so, we have to compute the marginal probability for answering correctly and wrong $p(x)$ and then also the conditional probability $p(x|l)$ first.

In [ ]:
# compute the marginal probability of a correct answer
# since we have a binary response, we can compute the marginal probability by taking the mean of the dataset
marginal_prob = np.mean(observations)

print(f'The marginal probability of a correct answer is: {marginal_prob}')

# compute the conditional probability of a correct answer given a sequence length
# we can do that by computing the mean of the dataset 
# first along the repition dimension -> This gives us the mean response given by each experimental unit for each sequence length
# second along the experimental unit dimension -> This gives us the mean response for each sequence length over all experimental units
conditional_prob = np.mean(observations, axis=2)
conditional_prob = np.mean(conditional_prob, axis=0)

print('The conditional probability of a correct answer given a sequence length is:')
for i, l in enumerate(sequence_lengths):
    print(f'{l}: {np.round(conditional_prob[i], 2)}')

Now we'll compute the information content of the possible responses.

In [ ]:
# compute the overall information content for correct and wrong answers
# use the computed marginal probability for that
information_content_correct = -np.log2(marginal_prob)
print(f'The information content of a correct answer is: {information_content_correct}')

information_content_wrong = -np.log2(1 - marginal_prob)
print(f'The information content of a wrong answer is: {information_content_wrong}')

# compute the conditional information content for correct and wrong answers
# use the computed conditional probabilities for that
conditional_information_content_correct = -np.log2(conditional_prob)
conditional_information_content_wrong = -np.log2(1 - conditional_prob)

print('The information content of correct and wrong answers given a sequence length are:')
print('Length\tCorrect\tWrong')
for i, l in enumerate(sequence_lengths):
    print(f'{l}\t{np.round(conditional_information_content_correct[i], 2)}\t{np.round(conditional_information_content_wrong[i], 2)}')

Are the information contents like you would have them expected to be?

You should find that the probability of an observation is negatively correlated with its information content.

Further, according to the response probabilities per sequence length, you should find that the shorter the presented sequence is, the higher is the information content of a wrong response and vice versa.

**What could this mean for our experimental setting? In which areas would you rather sample?**

### Entropy and conditional entropy

Here we are going to compute the entropy of the whole dataset, which is the expected information content per response.

To do so, we have to compute the average information content of each response.

In [ ]:
# compute the entropy of the response distribution
# use the formula from the lecture
entropy = -marginal_prob * np.log2(marginal_prob) - (1 - marginal_prob) * np.log2(1 - marginal_prob)
print(f'The entropy of the response distribution is: {entropy}')

What does this result tell you about the distribution and the expected information content?

In the binary case, an entropy $H(X)\neq 1$ indicates that the responses have an unbalanced probability and that the more likely one occures more frequently.

If the entropy is very unbalanced, that might be a hint for adapting our sampling strategy so we gather more samples of the less likely response.

Let's compute now the conditional entropy and check what this one tells us about our dataset.

Remember the formula from the lecture.

In [ ]:
# compute the conditional entropy as the sum of the conditional information content

# first, compute the joint probability of the response and the sequence length for each sample
# we can do that by finding the amount of occurrences of each combination of response and sequence length 
# and divide it by the total number of samples
joint_prob = np.zeros((len(sequence_lengths), 2))

for i, l in enumerate(sequence_lengths):
    joint_prob[i, 0] = np.sum(observations[:, i, :] == 0) / observations.size
    joint_prob[i, 1] = np.sum(observations[:, i, :] == 1) / observations.size

# now we are going to compute the conditional probability of the response given the sequence length
# we can do that by applying bayes rule: p(x, y) = p(x)*(y|x) = p(y)*(x|y)
prob_l = 1/len(sequence_lengths)
prob_x_given_l = np.zeros_like(joint_prob)
prob_x_given_l = joint_prob / prob_l 

# compute the conditional entropy as the sum of the conditional information content
# remember: in the case of a probability of 0, the log(0) would be -inf, which is not defined
# instead we can set the conditional entropy to 0 since 0*log(0) = 0
conditional_entropy = np.zeros_like(joint_prob)
for x in range(conditional_entropy.shape[1]):
    # correct and wrong
    for y in range(conditional_entropy.shape[0]):
        # sequence lengths
        if prob_x_given_l[y, x] == 0:
            conditional_entropy[y, x] = 0
        else:
            conditional_entropy[y, x] = joint_prob[y, x] * np.log2(prob_x_given_l[y, x])
conditional_entropy = -np.sum(conditional_entropy)

print(f'The conditional entropy for the whole dataset is: {conditional_entropy}')

What does this result tell us about our experiment and the collected data?

Remember that the conditional entropy tells us the remaining entropy of the responses given the conditions, which in our case are the sequence lengths.

Compare the overall response entropy with the conditional entropy. Did it change?

### Mutual information

In the last section you computed the conditional entropy, which is an indicator for remaining entropy of one random variable $X$ given another one $Y$.

Do you remember the Venn diagram from the lecture?

The mutual information $I(X,Y)$ was also computed with the conditional entropy and the overall entropy with a simple formula.
Compute the mutual information of the response and the sequence length.

In [ ]:
mutual_information_response_length = entropy - conditional_entropy
print(f'The mutual information between response and sequence length is: {mutual_information_response_length}')

How high is the mutual information compared to the overall entropy?

Can you think of a way to alter the mutual information? We could for example add some noise to the response data.

### Adding a notion of mystery to the data

In this step we are going to change our observations synthetically by flipping some of the bits and therefore add an unknown yet significant influence. 

This step simulates how the information characteristics change, when our chosen factors do not cover the cause-effect range to a satisfying degree.

You can do so by simply flipping the observations with a specified probability. Flipping means that 0 will become 1 and vice versa.

After modifying the dataset, re-run the previous cells from the information theory section and observe how the information characteristics change.

In [17]:
prob_flip = 0.3  # probability of flipping a response

idx = np.random.randint(0, observations.size, int(observations.size * prob_flip))  # get random indexes for the dataset
idx = np.unravel_index(idx, observations.shape)  # convert the indexes to the shape of the dataset

# we flip the responses for these indexes
observations[idx] = 1 - observations[idx]

## Further thoughts

### What could be limiting factors to our experiment?

How could the remaining high conditional entropy/low mutual information be explained?

Is our sample size the limiting factor or the experiment in general?

### Is this approach applicable to our response time experiment?

Recall the used equations.

How is the response time experiment different to the working memory experiment?

## Congratulations!

You've successfully finished the tutorial on the basics of information theory and how to apply them in the cognitive experimental context!

This was still a rather simple example regarding that our data had only one factor and the responses were binary.

When we are working e.g. with continuous variables things become way more complicated.

But for this case, there are already some popular libraries for optimal experimenting design and active learning which we are going to use in the next tutorial!